In [388]:
import numpy as np
import pandas as pd
import nltk 
import string
from nltk.corpus import stopwords
nltk.download('wordnet')
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [389]:
df=pd.read_csv("train_tweets.csv")

In [390]:
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [391]:
df['length']=df['tweet'].apply(len)

In [392]:
df.head()

,id,label,tweet,length
0,1,0,@user when a father is dysfunctional and is s...,102
1,2,0,@user @user thanks for #lyft credit i can't us...,122
2,3,0,bihday your majesty,21
3,4,0,#model i love u take with u all the time in ...,86
4,5,0,factsguide: society now #motivation,39


In [393]:
def text_processor(text):
  return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "",text.lower()).split())

In [394]:
df['cleaned tweets']=df['tweet'].apply(text_processor)
df.head()

,id,label,tweet,length,cleaned tweets
0,1,0,@user when a father is dysfunctional and is s...,102,when a father is dysfunctional and is so selfi...
1,2,0,@user @user thanks for #lyft credit i can't us...,122,thanks for lyft credit i cant use cause they d...
2,3,0,bihday your majesty,21,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,86,model i love u take with u all the time in ur
4,5,0,factsguide: society now #motivation,39,factsguide society now motivation


In [395]:
from nltk.stem import WordNetLemmatizer

In [396]:
stemmer=PorterStemmer()

In [397]:
X=df['cleaned tweets']
y=df['label']

In [398]:
def stemming(text):
  words=nltk.word_tokenize(text)
  words=[stemmer.stem(word) for word in words]
  words=" ".join(words)
  return words

In [399]:
count_class_0 ,count_class_1 = df.label.value_counts()
class_0 = df[df.label == 0]
class_1 =df[df.label == 1]

In [400]:

class_1_over = class_1.sample(count_class_0, replace=True)
train_over = pd.concat([class_0, class_1_over ], axis=0)

print('Random over-sampling:')
#print(df_test_under.sentiment.value_counts())
print(train_over.label.value_counts())
print(train_over.head())

Random over-sampling:
1    29720
0    29720
Name: label, dtype: int64
   id  label  ... length                                     cleaned tweets
0   1      0  ...    102  when a father is dysfunctional and is so selfi...
1   2      0  ...    122  thanks for lyft credit i cant use cause they d...
2   3      0  ...     21                                bihday your majesty
3   4      0  ...     86      model i love u take with u all the time in ur
4   5      0  ...     39                  factsguide society now motivation

[5 rows x 5 columns]


In [401]:
X=df['cleaned tweets'].apply(stemming)

In [402]:
y=df['label']


In [403]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [410]:
cvr=CountVectorizer(stop_words='english')
tfidf=TfidfTransformer()

In [406]:

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score


In [407]:

nb=PassiveAggressiveClassifier(random_state=101)


In [408]:
scores = []
cv = KFold(n_splits=10, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    
    X_train=cvr.fit_transform(X_train)
    X_test=cvr.transform(X_test)
    X_train=tfidf.fit_transform(X_train)
    X_test=tfidf.transform(X_test)
    nb.fit(X_train, y_train)
    pred=nb.predict(X_test)
    scores.append(f1_score(y_test,pred))


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train Index:  [ 3197  3198  3199 ... 31959 31960 31961] 

Test Index:  [   0    1    2 ... 3194 3195 3196]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [3197 3198 3199 ... 6391 6392 6393]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [6394 6395 6396 ... 9587 9588 9589]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [ 9590  9591  9592 ... 12783 12784 12785]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [12786 12787 12788 ... 15979 15980 15981]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [15982 15983 15984 ... 19175 19176 19177]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [19178 19179 19180 ... 22371 22372 22373]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [22374 22375 22376 ... 25567 25568 25569]
Train Index:  [    0     1     2 ... 31959 31960 31961] 

Test Index:  [25570 25571 25572 ... 28763 28764 28765]
T

In [409]:
print(scores)

[0.7199999999999999, 0.6880733944954128, 0.6602870813397128, 0.691304347826087, 0.6681614349775785, 0.6482412060301508, 0.6767895878524945, 0.6898047722342733, 0.6604215456674473, 0.6770601336302895]


In [374]:
df2=pd.read_csv('test_tweets.csv')
submit=pd.DataFrame()


In [375]:

X=df2['tweet']
df2['cleaned tweets']=df2['tweet'].apply(text_processor)


X=df2['cleaned tweets']
X=X.apply(stemming)
X=cvr.transform(X)
X=tfidf.transform(X)
predi=nb.predict(X)
submit['label']=predi


In [376]:
submit['id']=df2['id']

In [377]:
submit.head()
submit.to_csv("submission9.csv")

,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0
